# setup

In [19]:
import cirq
import numpy as np
from mitiq.benchmarks import generate_rb_circuits
from mitiq import MeasurementResult, Observable, PauliString, raw

# task

In [20]:
circuit = generate_rb_circuits(2,10)[0]

# noise model and executor

In [21]:
# apply depolarizing error and bit-flip error

def execute(circuit: cirq.Circuit, noise_level: float = 0.002, p0: float = 0.05) -> MeasurementResult:
    measurements = circuit[-1]
    circuit =  circuit[:-1]
    circuit = circuit.with_noise(cirq.depolarize(noise_level))
    circuit.append(cirq.bit_flip(p0).on_each(circuit.all_qubits()))
    circuit.append(measurements)

    simulator = cirq.DensityMatrixSimulator()

    result = simulator.run(circuit, repetitions=10000)
    bitstrings = np.column_stack(list(result.measurements.values()))
    return MeasurementResult(bitstrings)

# observable

In [26]:
from functools import partial

obs = Observable(PauliString("ZI"),PauliString("IZ"))
noisy = raw.execute(circuit,execute,obs)
ideal = raw.execute(circuit,partial(execute,noise_level = 0, p0 = 0), obs)
print("ideal value:", "{:.5f}".format(ideal.real))
print("Unmitigated value:", "{:.5f}".format(noisy.real))

ideal value: 2.00000
Unmitigated value: 1.34660


In [23]:
from mitiq import rem

p0 = p1 = 0.05
icm = rem.generate_inverse_confusion_matrix(2,p0,p1)
rem_executor = rem.mitigate_executor(execute,inverse_confusion_matrix=icm)

rem_result = obs.expectation(circuit,rem_executor)
print("Mitigated value obtained with REM:", "{:.5f}".format(rem_result.real))

Mitigated value obtained with REM: 1.49400


In [24]:
from mitiq import zne

# mitigate_executor : return modifieird version of input executor
zne_executor = zne.mitigate_executor(execute,observable=obs,scale_noise=zne.scaling.fold_global)
zne_result = zne_executor(circuit)
print("Mitigated value obtained with ZNE:", "{:.5f}".format(zne_result.real))

Mitigated value obtained with ZNE: 1.72220


In [25]:
combined_exevutor = zne.mitigate_executor(rem_executor,observable=obs,scale_noise=zne.scaling.fold_global)
combined_result = combined_exevutor(circuit)
print("Mitigated value obtained with REM + ZNE:", "{:.5f}".format(combined_result.real))

Mitigated value obtained with REM + ZNE: 2.06480
